In [ ]:
!pip install vaderSentiment

from google.colab import drive
from time import sleep
import json
import pandas as pd
import io
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tnrange, tqdm_notebook, tqdm
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt




from bs4.element import NamespacedAttribute
import requests
from bs4 import BeautifulSoup
import pandas as pd

drive.mount("/content/gdrive")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df=pd.read_csv('/content/gdrive/My Drive/capstone project/modified datasets/CLEAN_TWEETS_FILE.csv',low_memory=False)

In [ ]:
df=pd.read_csv('/content/gdrive/My Drive/capstone project/modified datasets/CLEAN_TWEETS_FILE.csv',low_memory=False)
analyze = SentimentIntensityAnalyzer()
compound = []
pos_pol=[]
neg_pol=[]
neu_pol=[]

for i,t in enumerate(tqdm(df['text'],position=0, leave=True)):
    vs = analyze.polarity_scores(str(t))

    compound.append(vs["compound"])
    pos_pol.append(vs["pos"])
    neg_pol.append(vs["neg"])
    neu_pol.append(vs["neu"])

df["compound"] = compound
df["pos"] = pos_pol
df["neg"] = neg_pol
df["neu"] = neu_pol

100%|██████████| 2830476/2830476 [06:17<00:00, 7505.51it/s]


In [ ]:
def getProfession(name):
  
  common_query="https://www.google.com/search?q="
  headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36'}
  name_query="+".join(name.split(" "));           # for example "Elon+Musk"
  user_query = name_query + "+profession"    
  URL = common_query + user_query
  page = requests.get(URL,headers=headers)
  soup = BeautifulSoup(page.content, 'html.parser')

  prof_data = soup.find_all(class_='bVj5Zb FozYP')        # Multiple profession, class name = 'bVj5Zb FozYP'
  if prof_data!=[]:
    for j in range(len(prof_data)):
      prof_data[j]=prof_data[j].string   
          
  elif(soup.find(class_='Z0LcW')!=None):    # Single profession, class name = 'Z0LcW',
    prof_data = soup.find(class_='Z0LcW')       
    prof_data=[prof_data.string]                        

  elif(soup.find(class_='hgKElc')!=None):
    prof_data = [soup.find(class_='hgKElc').find("b").string]                               

  else:                                 #for the professions that google doesn't have the result at the top, none in this case 
    prof_data=[]
   
  return prof_data



In [ ]:
def makeProfessionColumn(column):
  df['Profession'] = column

In [ ]:
profession_col=["-" for i in range(len(df.user_name))]
for i in range(len(df.user_name)):
  if df.iloc[i]["user_verified"]==True:
    if(getProfession(df.iloc[i]["user_name"])!=[]):
      profession_col[i]=getProfession(df.iloc[i]["user_name"])

In [ ]:
df['user_profession']=profession_col

In [ ]:
df.to_csv('/content/gdrive/My Drive/capstone project/modified datasets/TWEETS_SENTIMENT_PROFESSION.csv')


In [ ]:
df=pd.read_csv('/content/gdrive/My Drive/capstone project/modified datasets/TWEETS_SENTIMENT_PROFESSION.csv',low_memory=False)

In [ ]:
print(df['user_profession'].unique())

['-']
